In [2]:
import pandas as pd
import os
import sklearn
import numpy as np
from utils import split_dataset

Load data

In [3]:
covariates = pd.read_csv("./data/bigdata.csv", index_col="Dates", skiprows=5)
covariates.index = pd.to_datetime(covariates.index)

labels = pd.read_csv("./data/jkse.csv", skiprows=5, index_col="Dates")
labels.index = pd.to_datetime(labels.index)

labels = labels.rename(columns={"PX_LAST": "JKSE_PRICE"})
labels['PCT_CHANGE_20_JKSE'] = labels['JKSE_PRICE'].pct_change(periods=20) * 100

Split the dataset per ticker

In [4]:
covlist = split_dataset(covariates)

## Create technical covariates


### PE Band & P/E Ratio  
$$
 \text{P/E Ratio} = \frac{1}{\text{EARN\_YLD}} 
$$

* $PE Band_q $ is the q-th percentile of P/E Ratio in a 60 trading days rolling window

### Mean Average Convergence/Divergence (MACD)

- MACD is the difference of short term $\text{EMA}_{12}$ and long term $\text{EMA}_{26}$ (signal for momentum)
- MACD Signal is the signal line of 9 day EMA of the MACD
- MACD Histogram is the difference between the MACD line and the signal line

In [5]:
colnames = covlist[0].columns.tolist()
for cov in covlist:
    # unify column names
    cov.columns = colnames
for cov in covlist:
    # Volume = Turnover / Close Price
    cov['VOLUME'] = cov['TURNOVER'] / cov['PX_LAST']
    # Calculate percent change * 100
    cov['PCT_CHANGE_20'] = cov['PX_LAST'].pct_change(periods=20) * 100
    # Ratio 10/30 = mean volume ratio for the last 10 days / mean volume ratio for the last 30 days
    cov['VOL_RATIO_10_20'] = cov['VOLUME'].rolling(window=10).mean() / cov['VOLUME'].rolling(window=20).mean()
    cov['VOL_RATIO_20_40'] = cov['VOLUME'].rolling(window=20).mean() / cov['VOLUME'].rolling(window=40).mean()
    cov['VOL_RATIO_40_80'] = cov['VOLUME'].rolling(window=40).mean() / cov['VOLUME'].rolling(window=80).mean()
    cov['VOL_RATIO_80_120'] = cov['VOLUME'].rolling(window=80).mean() / cov['VOLUME'].rolling(window=120).mean()
    
    # PE Band
    cov['PE_Ratio'] = 1 / cov['EARN_YLD']
    win = 60  # Set the rolling window period
    cov['PE_Band_25'] = cov['PE_Ratio'].rolling(win).quantile(0.25)
    cov['PE_Band_50'] = cov['PE_Ratio'].rolling(win).quantile(0.50)
    cov['PE_Band_75'] = cov['PE_Ratio'].rolling(win).quantile(0.75)

    # Calculate the 12-day EMA of PX_LAST
    ema_12 = cov['PX_LAST'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-day EMA of PX_LAST
    ema_26 = cov['PX_LAST'].ewm(span=26, adjust=False).mean()

    # Calculate MACD
    cov['MACD'] = ema_12 - ema_26

    # Calculate the Signal line (9-day EMA of MACD)
    cov['MACD_Signal'] = cov['MACD'].ewm(span=9, adjust=False).mean()

    # Optionally, you can also calculate the MACD Histogram (the difference between MACD and Signal line)
    cov['MACD_Histogram'] = cov['MACD'] - cov['MACD_Signal']
    
    # Example: Momentum Indicator for various lags
    lags = [10, 20, 30, 60, 120]
    for lag in lags:
        cov[f'MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'TURNOVER_{lag}'] = cov['TURNOVER'].rolling(window=lag).mean()
        cov[f'PX_MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'PX_REVERSAL_{lag}'] = cov['PX_LAST'].shift(lag) / cov['PX_LAST']
        cov[f'VOLATILITY_{lag}'] = cov['PX_LAST'].rolling(window=lag).std()
        cov[f'VOLUME_STD_{lag}'] = cov['VOLUME'].rolling(window=lag).std()

In [6]:
# Remove duplicate indices if any
labels = labels[~labels.index.duplicated(keep='first')]

for i in range(len(covlist)):
    cov = covlist[i]
    cov = cov[~cov.index.duplicated(keep='first')]

    # Explicitly create a copy of the cov DataFrame to avoid SettingWithCopyWarning
    cov_copy = cov.copy()

    # Align the DataFrames on their indices (dates)
    aligned_df = labels.join(cov_copy[['PCT_CHANGE_20']], how='inner')

    # Calculate the difference and store it in cov_copy DataFrame
    cov_copy.loc[aligned_df.index, 'DELTA_20_CHANGE'] = aligned_df['PCT_CHANGE_20'] - aligned_df['PCT_CHANGE_20_JKSE']

    # Update the original DataFrame in covlist
    covlist[i] = cov_copy


In [8]:
tickernames = [col[:4] for col in pd.read_csv("./data/bigdata.csv", skiprows=3).columns if not col.startswith("Unnamed")]

/var/folders/5w/sk_z507j78j69yh904rh10440000gn/T/ipykernel_92329/2531172465.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [10]:
len(tickernames)

27

In [12]:
for i in range(len(covlist)):
    covlist[i]['Ticker'] = tickernames[i]

## This is for hmm clean

In [14]:
deltas = []
for cov in covlist:
    delta = cov[['DELTA_20_CHANGE']]
    deltas.append(delta)

In [15]:
delta_df = pd.concat(deltas, axis=1)
tickernames = [col[:4] for col in pd.read_csv("./data/bigdata.csv", skiprows=3).columns if not col.startswith("Unnamed")]
delta_df.columns = tickernames

/var/folders/5w/sk_z507j78j69yh904rh10440000gn/T/ipykernel_92329/2692577310.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [16]:
delta_df.drop(['MDKA', 'ICBP', 'ARTO', 'PGEO'], axis=1, inplace=True)
delta_df.dropna(inplace=True, axis=0, how='all')

In [17]:
delta_df.to_csv('./covset0/unnorm/hmm.csv')

In [18]:
def unique_index(df, suffix):
    df_copy = df.copy()
    new_index = [f"{date.strftime('%Y-%m-%d')}-{suffix}" for date in df.index]
    df_copy.index = new_index
    return df_copy

Create reindexed covlist + pool to find quantiles + insert index back to other covariates

In [19]:
covlist_reindex = []
for idx, cov in enumerate(covlist):
    covlist_reindex.append(unique_index(cov, idx))
    
pooled_df = pd.DataFrame()
for i, df in enumerate(covlist_reindex):
    pooled_df = pd.concat([pooled_df, df['DELTA_20_CHANGE']])

pooled_df['DELTA_20_QUINTILES'] = pd.qcut(pooled_df[0], q=5, labels=range(1, 6))

for i, df in enumerate(covlist_reindex):
    df['DELTA_20_QUINTILES'] = pooled_df.loc[df.index, 'DELTA_20_QUINTILES']
    covlist_reindex[i] = df

In [20]:
covlist[0].columns


Index(['EARN_YLD', 'PX_TO_BOOK_RATIO', 'PX_TO_SALES_RATIO', 'RETURN_ON_ASSET',
       'RETURN_COM_EQY', 'CUR_MKT_CAP', 'TRAIL_12M_NET_INC_GROWTH',
       'OPER_INC_GROWTH', 'VWAP_STANDARD_DEV', 'PX_LAST', 'TURNOVER', 'VOLUME',
       'PCT_CHANGE_20', 'VOL_RATIO_10_20', 'VOL_RATIO_20_40',
       'VOL_RATIO_40_80', 'VOL_RATIO_80_120', 'PE_Ratio', 'PE_Band_25',
       'PE_Band_50', 'PE_Band_75', 'MACD', 'MACD_Signal', 'MACD_Histogram',
       'MOMENTUM_10', 'TURNOVER_10', 'PX_MOMENTUM_10', 'PX_REVERSAL_10',
       'VOLATILITY_10', 'VOLUME_STD_10', 'MOMENTUM_20', 'TURNOVER_20',
       'PX_MOMENTUM_20', 'PX_REVERSAL_20', 'VOLATILITY_20', 'VOLUME_STD_20',
       'MOMENTUM_30', 'TURNOVER_30', 'PX_MOMENTUM_30', 'PX_REVERSAL_30',
       'VOLATILITY_30', 'VOLUME_STD_30', 'MOMENTUM_60', 'TURNOVER_60',
       'PX_MOMENTUM_60', 'PX_REVERSAL_60', 'VOLATILITY_60', 'VOLUME_STD_60',
       'MOMENTUM_120', 'TURNOVER_120', 'PX_MOMENTUM_120', 'PX_REVERSAL_120',
       'VOLATILITY_120', 'VOLUME_STD_120', '

Drop NaNs

In [21]:
for i in range(len(covlist_reindex)):
    covlist_reindex[i] = covlist_reindex[i].dropna()
    covlist_reindex[i] = covlist_reindex[i].reset_index(drop=False)

In [23]:
def rolling_window_save(train_window=252, test_window=60):
    os.makedirs('train_data', exist_ok=True)
    
    for cov in covlist_reindex:
        # Also removes labels 
        # cov.drop(['DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1, inplace=True)
        tdf = []
        vdf = []
        for start in range(0, len(cov) - train_window - test_window, train_window + test_window):
            end_train = start + train_window
            end_test = end_train + test_window
            
            train_df = cov.iloc[start:end_train]
            test_df = cov.iloc[end_train:end_test]
            
            tdf.append(train_df)
            vdf.append(test_df)
    
    return tdf, vdf

tdf, vdf = rolling_window_save(252, 60)

save data

In [33]:
valid.columns

Index(['index', 'EARN_YLD', 'PX_TO_BOOK_RATIO', 'PX_TO_SALES_RATIO',
       'RETURN_ON_ASSET', 'RETURN_COM_EQY', 'CUR_MKT_CAP',
       'TRAIL_12M_NET_INC_GROWTH', 'OPER_INC_GROWTH', 'VWAP_STANDARD_DEV',
       'PX_LAST', 'TURNOVER', 'VOLUME', 'PCT_CHANGE_20', 'VOL_RATIO_10_20',
       'VOL_RATIO_20_40', 'VOL_RATIO_40_80', 'VOL_RATIO_80_120', 'PE_Ratio',
       'PE_Band_25', 'PE_Band_50', 'PE_Band_75', 'MACD', 'MACD_Signal',
       'MACD_Histogram', 'MOMENTUM_10', 'TURNOVER_10', 'PX_MOMENTUM_10',
       'PX_REVERSAL_10', 'VOLATILITY_10', 'VOLUME_STD_10', 'MOMENTUM_20',
       'TURNOVER_20', 'PX_MOMENTUM_20', 'PX_REVERSAL_20', 'VOLATILITY_20',
       'VOLUME_STD_20', 'MOMENTUM_30', 'TURNOVER_30', 'PX_MOMENTUM_30',
       'PX_REVERSAL_30', 'VOLATILITY_30', 'VOLUME_STD_30', 'MOMENTUM_60',
       'TURNOVER_60', 'PX_MOMENTUM_60', 'PX_REVERSAL_60', 'VOLATILITY_60',
       'VOLUME_STD_60', 'MOMENTUM_120', 'TURNOVER_120', 'PX_MOMENTUM_120',
       'PX_REVERSAL_120', 'VOLATILITY_120', 'VOLUME_ST

In [34]:
valid.columns

Index(['index', 'EARN_YLD', 'PX_TO_BOOK_RATIO', 'PX_TO_SALES_RATIO',
       'RETURN_ON_ASSET', 'RETURN_COM_EQY', 'CUR_MKT_CAP',
       'TRAIL_12M_NET_INC_GROWTH', 'OPER_INC_GROWTH', 'VWAP_STANDARD_DEV',
       'PX_LAST', 'TURNOVER', 'VOLUME', 'PCT_CHANGE_20', 'VOL_RATIO_10_20',
       'VOL_RATIO_20_40', 'VOL_RATIO_40_80', 'VOL_RATIO_80_120', 'PE_Ratio',
       'PE_Band_25', 'PE_Band_50', 'PE_Band_75', 'MACD', 'MACD_Signal',
       'MACD_Histogram', 'MOMENTUM_10', 'TURNOVER_10', 'PX_MOMENTUM_10',
       'PX_REVERSAL_10', 'VOLATILITY_10', 'VOLUME_STD_10', 'MOMENTUM_20',
       'TURNOVER_20', 'PX_MOMENTUM_20', 'PX_REVERSAL_20', 'VOLATILITY_20',
       'VOLUME_STD_20', 'MOMENTUM_30', 'TURNOVER_30', 'PX_MOMENTUM_30',
       'PX_REVERSAL_30', 'VOLATILITY_30', 'VOLUME_STD_30', 'MOMENTUM_60',
       'TURNOVER_60', 'PX_MOMENTUM_60', 'PX_REVERSAL_60', 'VOLATILITY_60',
       'VOLUME_STD_60', 'MOMENTUM_120', 'TURNOVER_120', 'PX_MOMENTUM_120',
       'PX_REVERSAL_120', 'VOLATILITY_120', 'VOLUME_ST

In [38]:
valid = pd.concat(vdf, axis=0)
valid.index = valid['index']
valid.drop('index', axis=1, inplace=True)

In [40]:
valid.loc[:, ['PCT_CHANGE_20', 'PX_LAST', 'Ticker']]

,PCT_CHANGE_20,PX_LAST,Ticker
index,,,
2011-02-09-26,-13.852814,3980.0,PTBA
2011-02-10-26,-17.872340,3860.0,PTBA
2011-02-11-26,-16.916488,3880.0,PTBA
2011-02-14-26,-13.973799,3940.0,PTBA
2011-02-16-26,-15.869565,3870.0,PTBA
...,...,...,...
2024-02-23-26,2.334630,2630.0,PTBA
2024-02-26-26,0.389105,2580.0,PTBA
2024-02-27-26,0.000000,2590.0,PTBA


In [18]:
folder_name = 'covset0_pred'
os.makedirs(folder_name, exist_ok=True)
for i in range(len(tdf)):
    # tdf[i].to_csv(f'{folder_name}/train_{i}.csv', index=False)
    vdf[i].to_csv(f'{folder_name}/valid_{i}_pred.csv', index=False)